# 1. Imports

In [1]:
from src.configuration import Configuration
from src.fetch_data import DataFetcher
from aggregate_data import DataAggregator
from src.dim import Dims
from src.facts import Facts
from src.dicts import Dicts
import pandas as pd
import time

# 2. Variables

In [2]:
config = Configuration('src/config.cfg')
fetcher = DataFetcher(config)
DATAAGGREGATOR = DataAggregator(config, fetcher)
DIMS = Dims(config, fetcher)
DICTS = Dicts(config, fetcher)
FACTS = Facts(config, fetcher)

25/10/26 17:17:16 WARN Utils: Your hostname, MacBook-Pro-Grzegorz.local resolves to a loopback address: 127.0.0.1; using 192.168.0.3 instead (on interface en0)
25/10/26 17:17:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/26 17:17:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 3. Data

## 3.1 Driver related data

In [4]:
driver_last_5_races_result = DATAAGGREGATOR.get_last_races_result(5,"Race", "position")
time.sleep(5)
driver_last_5_quali_result = DATAAGGREGATOR.get_last_races_result(5,"Qualifying", "position")
time.sleep(5)
driver_last_5_avg_race_position = DATAAGGREGATOR.get_last_races_result(5, "Race", "avg")
time.sleep(5)
driver_last_5_std_race_position = DATAAGGREGATOR.get_last_races_result(5, "Race", "std")
time.sleep(5)
driver_last_5_avg_quali_position = DATAAGGREGATOR.get_last_races_result(5, "Qualifying", "avg")
time.sleep(5)
driver_last_5_std_quali_position = DATAAGGREGATOR.get_last_races_result(5, "Qualifying", "std")
time.sleep(5)
driver_points_gathered = DATAAGGREGATOR.get_racer_team_points("driver")
time.sleep(10)
driver_gap_to_teammate = DATAAGGREGATOR.calculate_gap_to_teammate()
time.sleep(5)
driver_gap_to_leader = DATAAGGREGATOR.calculate_gap_to_leader()
dict_world_champions = DICTS.dict_world_champions()
time.sleep(10)
pit_stops_efficiency = DATAAGGREGATOR.calculate_pit_stop_efficiency()

/Users/grzegorznaporowski/Desktop/Portfolio/F1/aggregate_data.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_filtered["points_gap_to_teammate"] = merged_filtered["points_gained"] - merged_filtered["points_gained_other"]


## 3.2 Team related data

In [5]:
team_points_gathered = DATAAGGREGATOR.get_racer_team_points("team")

## 3.3 Session Related Data

In [6]:
dim_session = DIMS.dim_sessions()
dim_session = dim_session[dim_session["session_name"]=="Race"]
needed_cols = ["key", "location", "year", "is_current_season","country_name", "date_start", "session_key"]
dim_session = dim_session[needed_cols]

In [7]:
needed_cols = ["driver_number","key","number_of_laps","dnf","dns","dsq","duration"]
fact_session_result = FACTS.fact_session_results()
race_duration = fact_session_result.merge(
    dim_session,
    on = "session_key",
    how="inner"
)

race_duration = race_duration[needed_cols]
race_duration["duration"] = race_duration["duration"].fillna(9999)

/var/folders/rd/z4g97_m5135b3yksvpxnxw1m0000gn/T/ipykernel_5309/3914122254.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  race_duration["duration"] = race_duration["duration"].fillna(9999)


# Joined Data

In [8]:
df = (
    dim_session
    .merge(driver_last_5_races_result, on="key", how="inner")
    .merge(driver_last_5_quali_result, on=["key", "driver_number"], how="inner", suffixes=["_race", "_quali"])
    .merge(driver_last_5_avg_race_position, on=["driver_number", "key"])
    .merge(driver_last_5_std_race_position, on=["driver_number", "key"])
    .merge(driver_last_5_avg_quali_position, on=["driver_number", "key"])
    .merge(driver_last_5_std_quali_position, on=["driver_number", "key"])
    .merge(driver_points_gathered[["driver_number", "key", "points_gained"]], on=["driver_number", "key"])
    .merge(driver_gap_to_teammate, on=["driver_number", "key"])
    .merge(driver_gap_to_leader, on=["driver_number", "key"])
    .merge(team_points_gathered, on=["driver_number", "key"])
    .merge(dict_world_champions,on="driver_number",how="left")
    .fillna({"titles_count": 0})
    .assign(world_champion=lambda x: x["titles_count"].astype(int))
    .merge(race_duration, on=["driver_number","key"])
    .merge(pit_stops_efficiency, on=["driver_number","key"])
)


In [11]:
df

,key,location,year,is_current_season,country_name,date_start,session_key,driver_number,position_race,last_race_pos_1,...,team_points_gained,titles_count,world_champion,number_of_laps,dnf,dns,dsq,duration,pit_duration_avg,pit_stop_count
0,Barcelona2023,Barcelona,2023,0,Spain,2023-06-04,9102,1,1.0,2.0,...,254.0,4.0,4,66.0,False,False,False,5277.940,22.25,2
1,Barcelona2023,Barcelona,2023,0,Spain,2023-06-04,9102,2,20.0,16.0,...,1.0,0.0,0,65.0,False,False,False,9999.000,23.15,2
2,Barcelona2023,Barcelona,2023,0,Spain,2023-06-04,9102,4,17.0,17.0,...,17.0,0.0,0,65.0,False,False,False,9999.000,27.50,3
3,Barcelona2023,Barcelona,2023,0,Spain,2023-06-04,9102,10,10.0,9.0,...,40.0,0.0,0,66.0,False,False,False,5351.470,23.45,2
4,Barcelona2023,Barcelona,2023,0,Spain,2023-06-04,9102,11,4.0,1.0,...,254.0,0.0,0,66.0,False,False,False,5313.752,21.60,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,Austin2025,Austin,2025,1,United States,2025-10-19,9888,43,17.0,18.0,...,19.0,0.0,0,55.0,False,False,False,9999.000,24.00,1
1045,Austin2025,Austin,2025,1,United States,2025-10-19,9888,44,4.0,12.0,...,302.0,7.0,7,56.0,False,False,False,5668.697,23.90,1
1046,Austin2025,Austin,2025,1,United States,2025-10-19,9888,63,6.0,3.0,...,319.0,0.0,0,56.0,False,False,False,5673.617,23.50,1
1047,Austin2025,Austin,2025,1,United States,2025-10-19,9888,81,5.0,2.0,...,642.0,0.0,0,56.0,False,False,False,5669.839,23.40,1
